# Bow vs TfIdf

In [1]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

In [2]:

mlflow.set_tracking_uri('https://dagshub.com/dhyanendra.manit/CI-CD-Mlops-mini-project.mlflow')
dagshub.init(repo_owner='dhyanendra.manit', repo_name='CI-CD-Mlops-mini-project', mlflow=True)


Accessing as dhyanendra.manit

Initialized MLflow to track repo "dhyanendra.manit/CI-CD-Mlops-mini-project"

Repository dhyanendra.manit/CI-CD-Mlops-mini-project initialized!

In [3]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)

<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_147690/3210576681.py:30: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [5]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

/tmp/ipykernel_147690/2032139550.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [ ]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                
                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                mlflow.log_param("preprocessing_steps", "lower_case, remove_stop_words, removing_numbers, removing_punctuations, removing_urls, lemmatization")
                
                # Log feature names
                if hasattr(vectorizer, 'get_feature_names_out'):
                    feature_names = vectorizer.get_feature_names_out()
                else:
                    feature_names = vectorizer.get_feature_names()
                
                # Save feature names to a file and log as artifact
                import pandas as pd
                feature_names_df = pd.DataFrame(feature_names, columns=["feature"])
                feature_names_path = f"feature_names_{algo_name}_{vec_name}.csv"
                feature_names_df.to_csv(feature_names_path, index=False)
                mlflow.log_artifact(feature_names_path, artifact_path="feature_names")
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                # Model logging compatible with DagsHub
                run_id = mlflow.active_run().info.run_id
                model_dir = f"model_{run_id}"
                mlflow.sklearn.save_model(model, model_dir)
                mlflow.log_artifacts(model_dir, artifact_path="model")
                
                # Log the train and test data as CSV artifacts
                import scipy.sparse
                import numpy as np
                # Convert sparse matrices to dense for logging (if needed)
                if scipy.sparse.issparse(X_train):
                    X_train_dense = X_train.todense()
                    X_test_dense = X_test.todense()
                else:
                    X_train_dense = X_train
                    X_test_dense = X_test
                train_df = pd.DataFrame(X_train_dense, columns=feature_names)
                train_df['label'] = np.array(y_train).reshape(-1)
                test_df = pd.DataFrame(X_test_dense, columns=feature_names)
                test_df['label'] = np.array(y_test).reshape(-1)
                train_path = f"train_{algo_name}_{vec_name}.csv"
                test_path = f"test_{algo_name}_{vec_name}.csv"
                train_df.to_csv(train_path, index=False)
                test_df.to_csv(test_path, index=False)
                mlflow.log_artifact(train_path, artifact_path="data")
                mlflow.log_artifact(test_path, artifact_path="data")
                
                # Log the current notebook file (if it exists)
                import os
                current_notebook = "exp2_bow_vs_tfidf.ipynb"
                if os.path.exists(current_notebook):
                    mlflow.log_artifact(current_notebook)
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
🏃 View run LogisticRegression with BoW at: https://dagshub.com/dhyanendra.manit/CI-CD-Mlops-mini-project.mlflow/#/experiments/1/runs/965e70053c5841808d089e463c044161
🧪 View experiment at: https://dagshub.com/dhyanendra.manit/CI-CD-Mlops-mini-project.mlflow/#/experiments/1
Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/dhyanendra.manit/CI-CD-Mlops-mini-project.mlflow/#/experiments/1/runs/9439f8f213fe4513abff76d04520b723
🧪 View experiment at: https://dagshub.com/dhyanendra.manit/CI-CD-Mlops-mini-project.mlflow/#/experiments/1
Algorithm: MultinomialNB, Feature Engineering: BoW
Accuracy: 0.7826506024096386
Precision: 0.7797619047619048